<a href="https://colab.research.google.com/github/leticiajsx/chatbot_sequential/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
# imports
import json
import pandas as pd
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder

# !pip install colorama
import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

In [ ]:
# JSON with bot intents and some categorized responses
# saved to a .json file
{
    "intents": [
        {
            "tag": "greeting",
            "patterns": [
                "Hi",
                "Hey",
                "Is anyone there?",
                "Hello",
                "Hay"
            ],
            "responses": [
                "Hello",
                "Hi",
                "Hi there"
            ]
        },
        {
            "tag": "goodbye",
            "patterns": [
                "Bye",
                "See you later",
                "Goodbye"
            ],
            "responses": [
                "See you later",
                "Have a nice day",
                "Bye! Come back again"
            ]
        },
        {
            "tag": "thanks",
            "patterns": [
                "Thanks",
                "Thank you",
                "That's helpful",
                "Thanks for the help"
            ],
            "responses": [
                "Happy to help!",
                "Any time!",
                "My pleasure",
                "You're most welcome!"
            ]
        },
        {
            "tag": "about",
            "patterns": [
                "Who are you?",
                "What are you?",
                "Who you are?"
            ],
            "responses": [
                "I.m Joana, your bot assistant",
                "I'm Joana, an Artificial Intelligent bot"
            ]
        },
        {
            "tag": "name",
            "patterns": [
                "what is your name",
                "what should I call you",
                "whats your name?"
            ],
            "responses": [
                "You can call me Joana.",
                "I'm Joana!",
                "Just call me as Joana"
            ]
        },
        {
            "tag": "help",
            "patterns": [
                "Could you help me?",
                "give me a hand please",
                "Can you help?",
                "What can you do for me?",
                "I need a support",
                "I need a help",
                "support me please"
            ],
            "responses": [
                "Tell me how can assist you",
                "Tell me your problem to assist you",
                "Yes Sure, How can I support you"
            ]
        },
        {
            "tag": "createaccount",
            "patterns": [
                "I need to create a new account",
                "how to open a new account",
                "I want to create an account",
                "can you create an account for me",
                "how to open a new account"
            ],
            "responses": [
                "You can just easily create a new account from our web site",
                "Just go to our web site and follow the guidelines to create a new account"
            ]
        },
        {
            "tag": "complaint",
            "patterns": [
                "have a complaint",
                "I want to raise a complaint",
                "there is a complaint about a service"
            ],
            "responses": [
                "Please provide us your complaint in order to assist you",
                "Please mention your complaint, we will reach you and sorry for any inconvenience caused"
            ]
        }
    ]
}

In [97]:
# extraction of the necessary data
with open('/content/intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [98]:
# transformation of tags into normalized encoding
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [99]:
# text data vectorization
# when using this class, all punctuation will be removed, transforming the texts into a sequence
# of words separated by spaces.
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [100]:
# neural network architecture using the Keras “Sequential” model.
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_7   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_21 (Dense)            (None, 16)                272       
                                                                 
 dense_22 (Dense)            (None, 16)                272       
                                                                 
 dense_23 (Dense)            (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
__________________________________________________

In [ ]:
# neural network training
epochs = 800
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

In [ ]:
# to save the trained model
model.save("chat_model")

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [103]:
def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: hey
1/1 [==============================] - 0s 97ms/step
ChatBot: Hello
User: you can help me?
1/1 [==============================] - 0s 31ms/step
ChatBot: Tell me your problem to assist you
User: who your name?
1/1 [==============================] - 0s 30ms/step
ChatBot: You can call me Joana.
User: thanks
1/1 [==============================] - 0s 41ms/step
ChatBot: Any time!
User: quit
